<a href="https://colab.research.google.com/github/DarkWorldCoder/LearningAI/blob/master/AI_Learning_with_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2023-06-05 01:27:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-05 01:27:05 (81.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,compare_historys,walk_through_dir

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"


--2023-06-05 01:31:16--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  34.1MB/s    in 48s     

2023-06-05 01:32:04 (32.1 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [4]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/macaroni_and_cheese'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/baby_back_ribs'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/spaghetti_bolognese'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/onion_rings'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/bread_pudding'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/seaweed_salad'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/creme_brulee'.
There are 0 director

In [5]:
import tensorflow as tf

IMG_SIZE = (224,224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=IMG_SIZE,
    shuffle=False
)


Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [6]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True
                                                         )


In [9]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [10]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.RandomZoom(0.2),

],name="data_augmentation")


In [13]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224,224,3),name="input_layer")
x = data_augmentation(inputs)
x = base_model(x,training=False)
x = layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),
                       activation="softmax",
                       name="output_layer"
                       )(x)

model = tf.keras.Model(inputs,outputs)


In [18]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]
              )

history_all_classes_10_percent = model.fit(train_data_all_10_percent,
          epochs=5,
          validation_data = test_data,
          validation_steps=int(0.15 * len(test_data)),
          callbacks=[checkpoint_callback]
          )



Epoch 1/5
237/237 [==============================] - 61s 227ms/step - loss: 1.3608 - accuracy: 0.6659 - val_loss: 1.6703 - val_accuracy: 0.5628
Epoch 2/5
237/237 [==============================] - 51s 214ms/step - loss: 1.2328 - accuracy: 0.6935 - val_loss: 1.6509 - val_accuracy: 0.5654
Epoch 3/5
237/237 [==============================] - 47s 198ms/step - loss: 1.1307 - accuracy: 0.7215 - val_loss: 1.6324 - val_accuracy: 0.5742
Epoch 4/5
237/237 [==============================] - 48s 199ms/step - loss: 1.0626 - accuracy: 0.7340 - val_loss: 1.6186 - val_accuracy: 0.5718
Epoch 5/5
237/237 [==============================] - 47s 198ms/step - loss: 0.9819 - accuracy: 0.7547 - val_loss: 1.6562 - val_accuracy: 0.5617


In [19]:

fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

790/790 [==============================] - 67s 84ms/step - loss: 1.5197 - accuracy: 0.5912


[1.519723653793335, 0.5911683440208435]

In [21]:
base_model.trainable = True

for layer in base_model.layers[:-5]:
  layer.trainable = True

model.compile(loss="categorical_crossentropy",
          optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=["accuracy"]
              )

fine_tune_epochs = 10

history_all_classes_fine_tune = model.fit(
    train_data_all_10_percent,
    epochs=fine_tune_epochs,
    validation_data = test_data,
    validation_steps = int(0.15 * len(test_data)),
    initial_epoch= history_all_classes_10_percent.epoch[:-1]
)


TypeError: ignored